In [ ]:
# Tutorium Programmieren (Prof. Dr. Ralf Gerlich)
# Aufgabenblatt 10: Personaldatenbank (Fortsetzung)
**Hinweis**: Für dieses Aufgabenblatts benötigen Sie Ihre Lösung vom vorangegangenen Aufgabenblatt als Grundlage. Die Lösungen der Aufgaben in diesem Aufgabenblatt werden im folgenden Aufgabenblatt erweitert.

## Einleitung: Sortieren von Listen

Die Funktion `sorted` erwartet eine Sequenz von Einträgen und liefert eine Liste zurück, die die Einträge in sortierter Reihenfolge enthält.

Probieren Sie es selbst:

In [1]:
l = [5,2,7,19,3]
sorted(l)

[2, 3, 5, 7, 19]

Auch eine umgekehrte Sortierung ist durch Angabe von `reverse=True` möglich:

In [12]:
l = [5,2,7,19,3]
sorted(l, reverse=True)

[19, 7, 5, 3, 2]

`sorted` nutzt dabei standardmäßig den `<=`-Operator, um die Objekte miteinander zu vergleichen. Steht ein solcher für die zu sortierenden Objekte nicht zur Verfügung oder soll eine andere als die standardmäßige Sortierung verwendet werden, so kann eine `key`-Funktion angegeben werden, die ein Objekt zurückliefert, nach dem sortiert werden soll.

Beispiel: Wir wollen Winkel nach ihrem Sinus-Wert sortieren:

In [6]:
from math import pi, sin
angles = [pi/2, 0, 3*pi/2]
sorted(angles, key=sin)

[4.71238898038469, 0, 1.5707963267948966]

Klassenmethoden können auch als Sortierfunktionen verwendet werden:

In [11]:
from math import sqrt
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def length(self):
        return sqrt(self.x**2+self.y**2)

    def __repr__(self):
        return f"Vector(x={self.x}, y={self.y})"

vectors = [Vector(1,1), Vector(2,0), Vector(0,-1)]
sorted(vectors, key=Vector.length)

[Vector(x=0, y=-1), Vector(x=1, y=1), Vector(x=2, y=0)]

## Aufgabe 1: Nächster Geburtstag
Ergänzen Sie die Klasse `Person` um eine Funktion `nächster_geburtstag`. Diese soll das Datum zurückliefern, an dem die Person zum nächsten Mal Geburtstag hat.

**Hinweise**:
- Um das heutige Datum zu ermitteln, können Sie die [Funktion `date.today()` aus der `datetime`-Bibliothek](https://docs.python.org/3/library/datetime.html#datetime.date.today) verwenden.
- `date`-Objekte können direkt per Vergleichsoperatoren `<`,  `<=`,  `>=` und `>` sowie `==` und `!=` miteinander verglichen werden. Dafür implementiert die Klasse `datetime.date` eine [Reihe von speziellen Methoden](https://docs.python.org/3/reference/datamodel.html#object.__lt__).

In [ ]:
from datetime import date
from typing import List
import os


class Person:
    def __init__(self, nachname, vorname, geburtsdatum):
        self._nachname: str = nachname
        self._vorname: str = vorname
        self._geburtsdatum: date = geburtsdatum
        self._nächster_geburtstag: date = self.rechne_naechsten_geburtstag(geburtsdatum)

    def rechne_naechsten_geburtstag(geburtstag):
        if geburtstag > date.today():
            return geburtstag
        else:
            return geburtstag.replace(year=date.today().year + 1)

    def __str__(self):
        return f"{self.vorname} {self.nachname}, {self.geburtsdatum}"

    @property
    def nachname(self):
        return self._nachname

    @property
    def vorname(self):
        return self._vorname

    @property
    def geburtsdatum(self):
        return self._geburtsdatum


class Personendatenbank:
    def __init__(self, personen):
        self._personen: List[Person] = personen

    def leeren(self):
        self._personen = []

    def einfügen(self, person):
        self._personen.append(person)

    def lade(self, datei):
        self._personen = []
        for l in datei.readlines():
            temp = l.split("$")
            self._personen.append(Person(temp[0], temp[1], temp[2]))

    def speichere(self, datei):
        for p in self.personen:
            datei.write(f"{p.nachname}${p.vorname}${p.geburtsdatum}$\n")

    def findePerson(self, name, vorname):
        for p in self.personen:
            if p.nachname.lower() == name.lower():
                if p.vorname.lower() == vorname.lower():
                    return p
        return None

    @property
    def personen(self):
        return self._personen


pdb = Personendatenbank([])

while True:
    action = int(
        input(
            """
        Guten Tag. Hier können sie ihre Mitarbeiter/Klienten/Sonstiges verwalten. Wählen sie eine Aktion aus!
        0 => Programm verlassen
        1 => Datenbank leeren
        2 => Person einfügen
        3 => Datenbank aus Datei laden
        4 => Datenbank in Datei speichern
        5 => Person finden
        """
        )
    )
    match action:
        case 0:
            break
        case 1:
            pdb.leeren()
            continue
        case 2:
            name = input("Nachname: \n")
            vorname = input("Vorname: \n")
            geburtsdatum = input("Geburtsdatum: \n")
            pdb.einfügen(Person(name, vorname, geburtsdatum))
            continue
        case 3:
            pfad = input("Welche Datei wollen sie laden?\n")
            if not os.path.isfile(pfad):
                print("Tut uns leid, diese Datei existiert nicht.\n")
                continue
            dbfile = open(pfad, "r")
            pdb.lade(dbfile)
            continue
        case 4:
            pfad = input("Wie soll die Datei heißen?\n")
            dbfile = open(pfad, "w")
            pdb.speichere(dbfile)
            continue
        case 5:
            name = input("Wie heißt ihre Person zum Nachnamen?\n")
            vorname = input("Und zum Vornamen?\n")
            person = pdb.findePerson(name, vorname)
            if person == None:
                print(
                    "Tut uns leid, diese Person existiert nicht in unserer Datenbank."
                )
                continue
            print(person.__str__())
            continue


## Aufgabe 2: Sortieren nach dem nächsten Geburtstag

Ergänzen Sie die Klasse `Personendatenbank` um eine Funktion `nächste_geburtstage`. Diese soll die Liste aller Personen zurückliefern, aber aufsteigend nach dem nächsten Geburtstag sortiert.

Ergänzen Sie dann Ihr Programm aus dem letzten Arbeitsblatt um eine Menüoption, mit der die fünf Personen ausgegeben werden können, die als nächstes Geburtstag haben. Die Reihenfolge soll dabei der Reihenfolge der Geburtstage entsprechen. Geben Sie zusätzlich das Alter aus, dass die Personen zu diesem Zeitpunkt erreicht haben werden.

Berücksichtigen Sie auch den Fall, in dem weniger als fünf Personen in Ihrer Datenbank enthalten sind.

In [ ]:
from datetime import date
from typing import List
import os


class Person:
    def __init__(self, nachname, vorname, geburtsdatum):
        self._nachname: str = nachname
        self._vorname: str = vorname
        self._geburtsdatum: date = geburtsdatum
        self._naechster_geburtstag: date = self.rechne_naechsten_geburtstag(
            geburtsdatum
        )

    def rechne_naechsten_geburtstag(geburtstag):
        if geburtstag > date.today():
            return geburtstag
        else:
            return geburtstag.replace(year=date.today().year + 1)

    def __str__(self):
        return f"{self.vorname} {self.nachname}, {self.geburtsdatum}"

    @property
    def nachname(self):
        return self._nachname

    @property
    def vorname(self):
        return self._vorname

    @property
    def geburtsdatum(self):
        return self._geburtsdatum

    @property
    def naechster_geburtstag(self):
        return self._naechster_geburtstag


class Personendatenbank:
    def __init__(self, personen):
        self._personen: List[Person] = personen

    def leeren(self):
        self._personen = []

    def einfügen(self, person):
        self._personen.append(person)

    def lade(self, datei):
        self._personen = []
        for l in datei.readlines():
            temp = l.split("$")
            self._personen.append(Person(temp[0], temp[1], temp[2]))

    def speichere(self, datei):
        for p in self.personen:
            datei.write(f"{p.nachname}${p.vorname}${p.geburtsdatum}$\n")

    def findePerson(self, name, vorname):
        for p in self.personen:
            if p.nachname.lower() == name.lower():
                if p.vorname.lower() == vorname.lower():
                    return p
        return None

    def naechste_geburtstage(self):
        return sorted(self.personen, key=self.personen.naechster_geburtstag)

    @property
    def personen(self):
        return self._personen


pdb = Personendatenbank([])

while True:
    action = int(
        input(
            """
        Guten Tag. Hier können sie ihre Mitarbeiter/Klienten/Sonstiges verwalten. Wählen sie eine Aktion aus!
        0 => Programm verlassen
        1 => Datenbank leeren
        2 => Person einfügen
        3 => Datenbank aus Datei laden
        4 => Datenbank in Datei speichern
        5 => Person finden
        6 => (max.) 5 nächste Geburtstagskinder auflisten
        """
        )
    )
    match action:
        case 0:
            break
        case 1:
            pdb.leeren()
            continue
        case 2:
            name = input("Nachname: \n")
            vorname = input("Vorname: \n")
            geburtsdatum = input("Geburtsdatum: \n")
            pdb.einfügen(Person(name, vorname, geburtsdatum))
            continue
        case 3:
            pfad = input("Welche Datei wollen sie laden?\n")
            if not os.path.isfile(pfad):
                print("Tut uns leid, diese Datei existiert nicht.\n")
                continue
            dbfile = open(pfad, "r")
            pdb.lade(dbfile)
            continue
        case 4:
            pfad = input("Wie soll die Datei heißen?\n")
            dbfile = open(pfad, "w")
            pdb.speichere(dbfile)
            continue
        case 5:
            name = input("Wie heißt ihre Person zum Nachnamen?\n")
            vorname = input("Und zum Vornamen?\n")
            person = pdb.findePerson(name, vorname)
            if person == None:
                print(
                    "Tut uns leid, diese Person existiert nicht in unserer Datenbank."
                )
                continue
            print(person.__str__())
            continue
        case 6:
            i = 0
            for p in self.naechste_geburtstage:
                i += 1
                print(p.__str__())
                print(
                    f"{p.vorname} wird ganze {p.naechster_geburtstag - p.geburtsdatum} Jahre alt!"
                )


## Aufgabe 3: Auszug aus Jupyter
Setzen Sie ein Projekt in PyCharm auf, in dem Ihr Personaldatenbank-Programm auch außerhalb von Jupyter ausgeführt werden kann. Lagern Sie die Klassen dabei in ein Modul `personendatenbank` aus. Das Hauptprogramm soll in einem gesonderten Modul `hauptprogramm` enthalten sein.

Testen Sie, ob Ihr Programm noch korrekt funktioniert.